In [25]:
import torch
import torch.nn as nn

torch.manual_seed(0)

softmax = nn.Softmax(dim=1)

logits = torch.randn(1, 3, requires_grad=True)
probs = softmax(logits)
labels = torch.empty(1, dtype=torch.long).random_(3)
labels_one_hot = torch.tensor([[0, 1, 0]], dtype=torch.long)

print((torch.log(probs) * labels_one_hot).sum())

print(logits, labels)
print(torch.log(probs))
# print(torch.log(probs)[1] * probs)

ce_loss = nn.CrossEntropyLoss()
print(ce_loss(logits, labels))

# print(logits, labels)


################ embeddings
# Define the number of categories and the number of dimensions for the embedding
num_categories = 10
embedding_dim = 5

# Define the embedding layer
embedding = nn.Embedding(num_categories, embedding_dim)

# Define the input as a PyTorch tensor of longs
input = torch.tensor([2, 3, 4, 5], dtype=torch.long)

# Pass the input through the embedding layer
output = embedding(input)
print(output)


tensor(-2.0033, grad_fn=<SumBackward0>)
tensor([[ 1.5410, -0.2934, -2.1788]], requires_grad=True) tensor([1])
tensor([[-0.1689, -2.0033, -3.8886]], grad_fn=<LogBackward0>)
tensor(2.0033, grad_fn=<NllLossBackward0>)
tensor([[ 0.9928, -0.1932, -0.3090,  0.5026, -0.8594],
        [ 0.7502, -0.1577,  1.4437,  0.2660,  0.1665],
        [ 0.8744, -0.1435, -0.1116, -0.6136,  1.2590],
        [ 2.0050,  0.0537,  0.6181, -0.4128, -0.8411]],
       grad_fn=<EmbeddingBackward0>)


In [11]:
import os
import numpy as np

path = '/data/manisha/datasets/fixed_policy/corner_start_fixed_hideouts/train'

# list of all files in the directory
files = sorted(os.listdir(path))
i = 20
file = np.load(os.path.join(path, files[i]), allow_pickle=True)
file['red_locations'][-1]


array([939, 542])

In [2]:
from torch.utils.data import Dataset, IterableDataset, DataLoader

class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

from itertools import islice
data = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

map_dataset = MyMapDataset(data)
loader = DataLoader(map_dataset, batch_size=4, shuffle=True)

for batch in loader:
    print(batch)

TypeError: __init__() got an unexpected keyword argument 'random_sampler'

In [27]:
import numpy as np

agents = [15, 1, 5]
x = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
y = np.repeat(x, agents, axis=0)
z = np.repeat(np.expand_dims(y, 0), 15, axis=0)
# print(z.shape)

a = np.random.rand(15, 21, 3)
# print(a.shape)

res = np.concatenate([a, z], axis=2)
# print(res.shape)

# print(res)

alpha = np.load("/nethome/sye40/PrisonerEscape/datasets/small_train/seed_0_known_27_unknown_26.npz", allow_pickle=True)
print(alpha["agent_dict"].item())
# a = np.array([, , 1])
# b = np.zeros((a.size, a.max()+1))
# b[np.arange(a.size),a] = 1
# print(b)

{'num_known_cameras': 55, 'num_unknown_cameras': 26, 'num_helicopters': 1, 'num_search_parties': 5}


In [9]:
import numpy as np

a = np.array([[0, 1, 0, 0], [0, 1, 1, 0], [0, 0, 0, 0]]) # timesteps x agents
locs = np.array([[2, 2], [3, 3], [4, 4]]) # timesteps x detected_locations

print(a.shape)
print(locs.shape)

# print(a * locs)
res = np.einsum("ij,ik->ijk", a, locs)
print(res.shape)
print(res)

(3, 4)
(3, 2)
(3, 4, 2)


In [23]:
# test to mae sure dataloader is functioning properly
path = "/nethome/sye40/PrisonerEscape/datasets/train_same/gnn_map_0_run_300_eps_0.1_norm"

from datasets.dataset import GNNPrisonerDataset
from datasets.old_gnn_dataset import LSTMGNNSequence
import numpy as np


seq_len = 16
step_length = 0
num_heads = 1

dataset_n = GNNPrisonerDataset(path, seq_len, num_heads, step_length, 
        include_current=False, 
        multi_head = False, 
        one_hot=False,
        timestep=False, 
        detected_location=False)

path_single = "/nethome/sye40/PrisonerEscape/datasets/concatted/train.npz"
np_file = np.load(path_single, allow_pickle=True)
future_step = step_length * num_heads
dataset = LSTMGNNSequence(
    np_file["agent_observations"], 
    np_file["hideout_observations"], 
    np_file["timestep_observations"],
    np_file["red_locations"], 
    np_file["dones"], 
    seq_len, future_step)

x_n, y_n = dataset_n[0]
x, y = dataset[0]

print(len(dataset_n))
print(len(dataset))

for i in range(len(dataset_n)):
    x_n, y_n = dataset_n[i]
    x, y = dataset[i]

    for j in range(3):

        if not np.allclose(x_n[j], np.squeeze(x[j])):
            print(i, j)
            # print(x_n[j]), x[j]
    # print(len(x_n))

96732
96732


In [22]:
# convert new format to old format
import os

total_agents = []
total_hideouts = []
total_timesteps = []
total_red_locations = []
total_dones = []

# folder_path = "/nethome/sye40/PrisonerEscape/datasets/test_same/gnn_map_0_run_100_eps_0.1_norm"
folder_path = "/nethome/sye40/PrisonerEscape/datasets/train_same/gnn_map_0_run_300_eps_0.1_norm"
# folder_path = "/nethome/sye40/PrisonerEscape/datasets/debug"
for file_name in os.listdir(folder_path):
    np_file = np.load(os.path.join(folder_path, file_name), allow_pickle=True)
    agent_o = np.squeeze(np_file["agent_observations"])
    hideout_o = np_file["hideout_observations"]
    t_o = np_file["timestep_observations"]
    r_o = np_file["red_locations"]/2428
    d_o = np_file["dones"]

    total_agents.append(agent_o)
    total_hideouts.append(hideout_o)
    total_timesteps.append(t_o)
    total_red_locations.append(r_o)
    total_dones.append(d_o)

# print(np.concatenate(total_agents, axis=0).shape)
np.savez("/nethome/sye40/PrisonerEscape/datasets/concatted/train.npz", 
            agent_observations=np.concatenate(total_agents, axis=0),
            hideout_observations=np.concatenate(total_hideouts, axis=0),
            timestep_observations=np.concatenate(total_timesteps, axis=0),
            red_locations=np.concatenate(total_red_locations, axis=0), 
            dones=np.concatenate(total_dones, axis=0)
            )
    


(96732, 83, 3)
